[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1MjzUke5SDxffqmpruiUzfdBLR6gKbCBM)

<br>

< [5. Modele](PD_modele.ipynb) | [7. Rozwiązanie Bazowe.](PD_rozw_bazowe.ipynb) >


## [6. Rozwiązania optymalizujące proces trenowania.]()

Zmiany poprawiające proces uczenia zaimpementowane w [[1] reid-strong-baseline -github](https://github.com/michuanhaohao/reid-strong-baseline) zostały zaprezentowane na grafice umieszczonej poniżej.
<br><br>
<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/reidTricks.jpg?raw=true" alt="drawing" width="400"/>
<br><br>


### 6.1 Rozgrzewanie kroku uczenia (Warmup Learning Rate)

Learning Rate ma bezpośredni wpływ na to jak szybko model zmienia swoje wagi. W celu wygenerowanai statystyk ułatwiających manipulowaniem parameterem Learning Rate przez optymalizator, statuje się mechanizm nazywany "Rozgrzewaniem kroku uczenia". Polega on na poświęceniu kilku pierwszych epok na stworzeniu statystyk przy ustawionym domyślnie kroku uczenia na niskim poziomie. W praktyce zaimplementowano to w spoósb:
 
 <img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/learning-rate.png?raw=true" alt="drawing" width="400"/>
<br><br>

### 6.2 Losowo wycinanie (Random Erasing Augumentation)

Losowe wycinanie fragmentów obrazów jest powszechnie stosowanym sposobem augumetacji danych. Ma on na celu uniezależnienie modelu na zmiany, które nie powinny być ważnymi cechami obiektu na ktorego podstawie generowany jest jego embeding.


Przykład realizowany na opisanym powyżej datasecie Martek1501:

 <img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/random_erasing.png?raw=true" alt="drawing" width="400"/>
<br><br>

### 6.3 Wygładzanie adnotacji (Label Smoothing)

Ponieważ adnotacja danych bierze udział w przeliczaniu funkcji strat jako jeden z jej składników, problemem staje się przetrenowanie modelu w kontekście wykrywania właśnie adnotacji. Metodą na prewencję takiej sytuacji jest użycie "Wygładzania adnotacji". Polega to rozłożeniu niewielkiej części prawdopodobieństwa na inne klasy. W przypadku rozpatrywanym w tej pracy na bazie zbioru danych Markets1501 inną klasą jest id innej osoby. 

<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/label_smoothing.png?raw=true" alt="drawing" width="400"/>

In [ ]:
from tabulate import tabulate
import numpy as np

g_truth = {"Alice":0,'Bob': 0,'Frank':1,'Mari': 0,'Eve': 0}

def pretty_display(var_list):
  from prettytable import PrettyTable
  t = PrettyTable(['Id_name', 'Ground_truth'])
  for i in var_list:
    t.add_row([i,var_list[i]])
  return t

reference_value= pretty_display(g_truth)

def label_smoothing(g_truth, eps):
  g_truth_len = len(g_truth)
  smoothed_list:dict ={}
  for i in g_truth:
    if g_truth[i] == 1:
      smoothed_list[i] = 1-((g_truth_len-1)*eps/g_truth_len)
    else:
      smoothed_list[i] = eps/g_truth_len
  return smoothed_list

print(reference_value)
print("////// Label Smoothing //////")
reference_value_after_label_smoothing = pretty_display(label_smoothing(g_truth,0.05))
print(reference_value_after_label_smoothing) 


+---------+--------------+
| Id_name | Ground_truth |
+---------+--------------+
|  Alice  |      0       |
|   Bob   |      0       |
|  Frank  |      1       |
|   Mari  |      0       |
|   Eve   |      0       |
+---------+--------------+
////// Label Smoothing //////
+---------+--------------+
| Id_name | Ground_truth |
+---------+--------------+
|  Alice  |     0.01     |
|   Bob   |     0.01     |
|  Frank  |     0.96     |
|   Mari  |     0.01     |
|   Eve   |     0.01     |
+---------+--------------+


### 6.4 Zmiana ostatniego warstwy konwolucyjnej (Last Stride)

Jedną z potrzeb jakie adresują wartwy konwolucyjne to agregacja wymiarów cech rozwiniętych przez poprzednie warstwy. Modyfikacja dotycząca tego punktu polega na ograniczeniu agregacji cech ostatniej warstwy i pozostawienia wiekszej reprezentacji cech głębokich sieci stanowiącej backbone trenowanego modelu. 

### 6.5 BBNeck

Dużą część pracy jaką poświęcili twórcy framework-u użytego w tej pracy, stanowiła zmiana opisana jako BBNeck.


<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/bnn-modify.png?raw=true" alt="drawing" width="400"/>


Jak zaprezentowano na załączonej ilustracji zaczerpniętej z pracy [[6]](https://arxiv.org/pdf/1903.07071.pdf) w przypadku a) będącym typowym podejściem ID loss i triplet loss działają na tym samej warstwie cech głębokich (embedding-ów). W takiej sytuacji lepiej sprawdza się przeliczenie dystansu cosinusowego czyli porównanie kierunków wyznaczonych przez wektory w przestrzeni embeddingów. Natomiast w przypadku b) wykorzystanie odległości euklidesowej w przestrzeni wektorów embeddingów sprzyja zagęszczaniu wyników wewnątrz klasową oraz zwiększanie odległości między zbiorami należacymi do innych klas. Ponieważ zagęszczanie wyników wewnątrz jednej klasy może zdecydowanie szybciej minimalizować funkcję strat triple loss, niż zwiększania odległości między klasowej wprowadzono kolejną warstwę cech głębokich. Kolejna warstwa jest znormalizowaną warstwą cech głębokich (embeddingów). Idea stojąca za takim krokiem polega na wymuszeniu stałej długości wektorów cech dla różnych klas. Zwiększa to znaczenie zwiększania odległości między różnymi klasami, co prowadzi do ograniczenia problemu minimalizacji międzyklasowej "kosztem" maksymalizacji odległości międzyklasowej. Idę bardzo dobrze ilustruje grafika zawarta w pracy [[6]](https://arxiv.org/pdf/1903.07071.pdf)

<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/bnn-modify-2.png?raw=true" alt="drawing" width="800"/>

### 6.6 Center Loss

To rozwiązanie dodające do fukcji strat bezpośredni człon odpowidzialny za penalizowanie dużych rozbieżności w obrębie jednej klasy. Formuła na przeliczenie takiego członu zaprezentowano poniżej:

<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/center-loss.png?raw=true" alt="drawing" width="400"/>

W rezultacie funkcja strat posiada trzy człony zaprazentowane poniżej:

<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/loss-fun.png?raw=true" alt="drawing" width="400"/>

### 6.7 Podsumowanie

W pracy [[6]](https://arxiv.org/pdf/1903.07071.pdf) zaprezentowano wyniki jakie dają użycie poszczególnych usprawnienień opisanych powyżej. To zestawienie wygląda jak poniżej:

<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/trick-summarization.png?raw=true" alt="drawing" width="400"/>

Jest to unikatowa wartość jaką wprowadza praca [[6]](https://arxiv.org/pdf/1903.07071.pdf). Daję ona wiarygone informację jaką jakościową zmianę można spodziewać się z użycia każdego z usprawnień i zaplanowanie wiarygodnego porównania zmian wprowadzanych do modeli. 

## Nawigacja



<br>

< [5. Modele](PD_modele.ipynb) | [7. Rozwiązanie Bazowe.](PD_rozw_bazowe.ipynb) >
